In [4]:
import sys
print(sys.executable)
print(sys.version)


c:\Users\AdrienSourdille\Coral_Bleaching_Live_Map\.venv\Scripts\python.exe
3.12.5 (tags/v3.12.5:ff3bc82, Aug  6 2024, 20:45:27) [MSC v.1940 64 bit (AMD64)]


In [ ]:
import requests
from datetime import datetime, timedelta

def fetch_sst_anomaly():
    # Get yesterday's date at 12:00 UTC (NOAA data timestamp)
    yesterday = datetime.utcnow().date() - timedelta(days=1)
    timestamp = yesterday.strftime("%Y-%m-%dT12:00:00Z")

    dataset_id = "noaacrwsstanomalyDaily"
    variable = "sea_surface_temperature_anomaly"

    # Correct URL format with slicing for time, lat, lon:
    # [startTime:step:endTime][startLat:step:endLat][startLon:step:endLon]
    url = (
        f"https://coastwatch.noaa.gov/erddap/griddap/{dataset_id}.csv?"
        f"{variable}%5B({timestamp}):1:({timestamp})%5D"
        f"%5B(13.025):1:(-59.975)%5D"
        f"%5B(78.025):1:(179.975)%5D"
    )

    print(f"Fetching data from {url}")
    response = requests.get(url)
    response.raise_for_status()  # Raise error if request failed

    # Save CSV locally
    with open(r"C:\Users\AdrienSourdille\Coral_Bleaching_Live_Map\data\raw\latest_sst_anomaly.csv", "wb") as f:
      f.write(response.content)

    print("Data saved to data/raw/latest_sst_anomaly.csv")

if __name__ == "__main__":
    fetch_sst_anomaly()


C:\Users\AdrienSourdille\AppData\Local\Temp\ipykernel_6180\52661120.py:6: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  yesterday = datetime.utcnow().date() - timedelta(days=1)


Fetching data from https://coastwatch.noaa.gov/erddap/griddap/noaacrwsstanomalyDaily.csv?sea_surface_temperature_anomaly%5B(2025-06-17T12:00:00Z):1:(2025-06-17T12:00:00Z)%5D%5B(13.025):1:(-59.975)%5D%5B(78.025):1:(179.975)%5D
Data saved to data/sst_anomaly.csv


In [17]:
import pandas as pd

def describe_sst_data(filepath=r"C:\Users\AdrienSourdille\Coral_Bleaching_Live_Map\data\raw\latest_sst_anomaly.csv"):
    # Load CSV (skip first rows if they are metadata, adjust skiprows if needed)
    df = pd.read_csv(filepath, skiprows=1)  # ERDDAP usually has 1 header row before data

    print("Columns:", df.columns.tolist())
    print("\nSample rows:")
    print(df.head())

    print("\nSummary statistics:")
    print(df.describe())

    print("\nData types:")
    print(df.dtypes)

if __name__ == "__main__":
    describe_sst_data()


Columns: ['UTC', 'degrees_north', 'degrees_east', 'degree_C']

Sample rows:
                    UTC  degrees_north  degrees_east  degree_C
0  2025-06-17T12:00:00Z         13.025        78.025       NaN
1  2025-06-17T12:00:00Z         13.025        78.075       NaN
2  2025-06-17T12:00:00Z         13.025        78.125       NaN
3  2025-06-17T12:00:00Z         13.025        78.175       NaN
4  2025-06-17T12:00:00Z         13.025        78.225       NaN

Summary statistics:
       degrees_north  degrees_east      degree_C
count   2.980440e+06  2.980440e+06  2.569294e+06
mean   -2.347500e+01  1.290000e+02  7.684449e-01
std     2.108772e+01  2.944487e+01  6.563551e-01
min    -5.997500e+01  7.802500e+01 -8.190000e+00
25%    -4.172500e+01  1.035125e+02  4.000000e-01
50%    -2.347500e+01  1.290000e+02  7.600000e-01
75%    -5.225000e+00  1.544875e+02  1.140000e+00
max     1.302500e+01  1.799750e+02  4.630000e+00

Data types:
UTC               object
degrees_north    float64
degrees_east     floa